# pandas的索引index的用处
把数据存储到普通的columns列也能用于数据查询，那使用index有什么好处？

index的用途总结：
1. 更方便的数据查询
2. 使用index可以获取性能提升
3. 自动的数据对齐功能
4. 更多更强大的数据结构支撑

In [1]:
import pandas as pd

df = pd.read_csv('./data/ml-latest-small/ratings.csv')

In [2]:
df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
df.count()

userId       100836
movieId      100836
rating       100836
timestamp    100836
dtype: int64

## 1 使用index查询数据

In [4]:
# drop=False: 让索引列还保持在columns
df.set_index('userId', inplace=True, drop=False)

In [5]:
df.head()

,userId,movieId,rating,timestamp
userId,,,,
1,1,1,4.0,964982703
1,1,3,4.0,964981247
1,1,6,4.0,964982224
1,1,47,5.0,964983815
1,1,50,5.0,964982931


In [7]:
df.index

Index([  1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
       ...
       610, 610, 610, 610, 610, 610, 610, 610, 610, 610],
      dtype='int64', name='userId', length=100836)

In [8]:
# 使用index的查询方法
df.loc[500].head(5)

,userId,movieId,rating,timestamp
userId,,,,
500,500,1,4.0,1005527755
500,500,11,1.0,1005528017
500,500,39,1.0,1005527926
500,500,101,1.0,1005527980
500,500,104,4.0,1005528065


In [9]:
# 传统的columns的condition查询方法
df.loc[df['userId']==500].head()

,userId,movieId,rating,timestamp
userId,,,,
500,500,1,4.0,1005527755
500,500,11,1.0,1005528017
500,500,39,1.0,1005527926
500,500,101,1.0,1005527980
500,500,104,4.0,1005528065


## 使用index会提升查询性能

- 如果index是唯一的，pandas会使用哈希表优化，查询性能为O(1)
- 如果index不是唯一的，但是有序，pandas会使用二分查找算法查找，性能为O(logN)
- 如果index是完全随机的，那么每次查询都要扫描全表，查询性能为O(N)

实验1：完全随机的顺序查询

In [10]:
# 将数据打散

from sklearn.utils import shuffle
df_shuffle = shuffle(df)

In [11]:
df_shuffle.head()

,userId,movieId,rating,timestamp
userId,,,,
588,588,296,5.0,839316050
232,232,46578,4.0,1166916648
474,474,5013,3.5,1129579311
290,290,2160,4.0,974938393
184,184,193565,3.5,1537098554


In [12]:
# 索引是否递增
df_shuffle.index.is_monotonic_increasing

False

In [13]:
df_shuffle.index.is_unique

False

In [14]:
# 计时，查询id==500的性能
%timeit df_shuffle.loc[500]

200 µs ± 12.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


# 实验2：将index排序后查找

In [15]:
df_sorted = df_shuffle.sort_index()

In [16]:
df_sorted.head()

,userId,movieId,rating,timestamp
userId,,,,
1,1,804,4.0,964980499
1,1,1408,3.0,964982310
1,1,3489,4.0,964981775
1,1,940,5.0,964982176
1,1,2502,5.0,964981278


In [18]:
# 索引是否递增
df_sorted.index.is_monotonic_increasing

True

In [19]:
df_shuffle.index.is_unique

False

In [20]:
%timeit df_shuffle.loc[500]

192 µs ± 16 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## 使用index自动对齐数据

包括series和dataframe

In [28]:
s1 = pd.Series([1,2,3], index=list('abc'))

In [29]:
s1

a    1
b    2
c    3
dtype: int64

In [30]:
s2 = pd.Series([4,5,6], index=list('bcd'))

In [31]:
s2

b    4
c    5
d    6
dtype: int64

In [32]:
s1 + s2

a    NaN
b    6.0
c    8.0
d    NaN
dtype: float64

## 使用index更多更强大的数据结构支持

1. CategoricalIndex,基于分类数据的index，提升性能；
2. MultiIndex,多维索引，用于groupby多维聚合后结果等；
3. DatetimeIndex,时间类型索引，强大的日期和时间的方法支持。